In [ ]:
!pip install snowflake-snowpark-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 419.7/419.7 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 kB 3.7 MB/s eta 0:00:00
  Attempting uninstall: platformdirs
    Found existing installation: platformdirs 4.2.0
    Uninstalling platformdirs-4.2.0:
      Successfully uninstalled platformdirs-4.2.0


In [ ]:
from snowflake.snowpark import Session
from snowflake.snowpark import functions as F
from snowflake.snowpark.functions import col, year, month, count, when, quarter
from snowflake.snowpark.functions import  concat, to_date
from snowflake.snowpark.functions import avg, stddev_pop, corr
import numpy as np
import pandas as pd
from snowflake.snowpark.exceptions import SnowparkSQLException
from snowflake.snowpark.functions import col, year, month, count, when, concat_ws, to_date, lpad
from google.colab import files

In [ ]:
parameters = {
...    "account": "YEZEPEO-DATASCIENCE",
...    "user": "VNSHARED",
...    "password": "Columbia2024!",
...    "role": "SYSADMIN", # optional
...    "warehouse": "ANALYSIS", # optional
...   "database": "MEIXI_DATA", # optional
...   "schema":"PUBLIC",  # optional
...  }
session= Session.builder.configs(parameters).create()

In [ ]:
# Monthly aggregation

In [ ]:
df_v=session.sql('select * from CONSOLIDATED_COPY ')
df_v = df_v.select([F.col(col_name).alias(col_name.replace('"', '')) for col_name in df_v.columns])
df_v = df_v.select("FECHAEMISION", "STATUSCOMPENSACION")

In [ ]:
# Extract year and month from FECHAEMISION
df_v = df_v.withColumn("YEAR", year(col("FECHAEMISION"))) \
           .withColumn("MONTH", month(col("FECHAEMISION")))

# Group by year and month
grouped_df = df_v.groupBy("YEAR", "MONTH")

# Calculate the likelihood of default
df_monthly_aggregated = grouped_df.agg(
    (count(when(col("StatusCompensacion").isin(["Impaga", "Castigado"]), col("StatusCompensacion"))) / count("*")).alias("LikelihoodOfDefault")
)

df_monthly_aggregated= df_monthly_aggregated.orderBy("YEAR", "MONTH")

In [ ]:
#df_monthly_aggregated.to_pandas()

,YEAR,MONTH,LIKELIHOODOFDEFAULT
0,2006,1,0.019363
1,2006,2,0.017998
2,2006,3,0.036930
3,2006,4,0.036691
4,2006,5,0.026561
...,...,...,...
212,2023,9,0.293259
213,2023,10,0.267782
214,2023,11,0.360104
215,2023,12,0.439466


In [ ]:
# Upload as table to Snowflake
table_name = 'monthly_default'

# Write the DataFrame to a new table in Snowflake
df_monthly_aggregated.write.mode('overwrite').save_as_table(table_name)

In [ ]:
# Yearly aggregation

In [ ]:
# Extract year from FECHAEMISION
df_v = df_v.withColumn("YEAR", year(col("FECHAEMISION")))

# Group by year
grouped_df = df_v.groupBy("YEAR")

# Calculate the likelihood of default
df_yearly_aggregated = grouped_df.agg(
    (count(when(col("StatusCompensacion").isin(["Impaga", "Castigado"]), col("StatusCompensacion"))) / count("*")).alias("LikelihoodOfDefault")
)

# Order by year
df_yearly_aggregated = df_yearly_aggregated.orderBy("YEAR")


In [ ]:
df_yearly_aggregated.to_pandas()

,YEAR,LIKELIHOODOFDEFAULT
0,2006,0.032722
1,2007,0.048319
2,2008,0.057514
3,2009,0.065448
4,2010,0.094427
5,2011,0.101237
6,2012,0.107576
7,2013,0.068015
8,2014,0.074191
9,2015,0.081544


In [ ]:
# Upload as table to Snowflake
table_name = 'yearly_default'

# Write the DataFrame to a new table in Snowflake
df_yearly_aggregated.write.mode('overwrite').save_as_table(table_name)

In [ ]:
# Aggregate Quarterly

In [ ]:
# Extract year and quarter from FECHAEMISION
df_v = df_v.withColumn("YEAR", year(col("FECHAEMISION"))) \
           .withColumn("QUARTER", quarter(col("FECHAEMISION")))

# Group by year and quarter
grouped_df = df_v.groupBy("YEAR", "QUARTER")

# Calculate the likelihood of default
df_quarterly_aggregated = grouped_df.agg(
    (count(when(col("StatusCompensacion").isin(["Impaga", "Castigado"]), col("StatusCompensacion"))) / count("*")).alias("LikelihoodOfDefault")
)

# Order by year and quarter
df_quarterly_aggregated = df_quarterly_aggregated.orderBy("YEAR", "QUARTER")


In [ ]:
df_quarterly_aggregated.to_pandas()

,YEAR,QUARTER,LIKELIHOODOFDEFAULT
0,2006,1,0.030029
1,2006,2,0.031522
2,2006,3,0.035340
3,2006,4,0.032226
4,2007,1,0.042080
...,...,...,...
68,2023,1,0.213754
69,2023,2,0.234664
70,2023,3,0.273729
71,2023,4,0.341785


In [ ]:
# Upload as table to Snowflake
table_name = 'quarterly_default'

# Write the DataFrame to a new table in Snowflake
df_quarterly_aggregated.write.mode('overwrite').save_as_table(table_name)

Merging Tables: General CPI, monthly

In [ ]:
df_cpi = session.sql('select * from CPI_COPY')
df_cpi = df_cpi.select([F.col(col_name).alias(col_name.replace('"', '')) for col_name in df_cpi.columns])
general_cpi_df = df_cpi[df_cpi['Descripción series'] == 'IPC general (empalme BCCh)\xa0']

df_d=session.sql('select * from MONTHLY_DEFAULT ')
df_d = df_d.select([F.col(col_name).alias(col_name.replace('"', '')) for col_name in df_d.columns])

In [ ]:
general_cpi_df=general_cpi_df.to_pandas()
df_d=df_d.to_pandas()

In [ ]:
# Convert 'Date (#)' in general_cpi_df to datetime
general_cpi_df['Date (#)'] = pd.to_datetime(general_cpi_df['Date (#)'])

# Make sure YEAR and MONTH in df_d are strings and MONTH is two digits
df_d['YEAR'] = df_d['YEAR'].astype(str)
df_d['MONTH'] = df_d['MONTH'].astype(str).str.zfill(2)

# Combine YEAR and MONTH into a DATE column in df_d
df_d['DATE'] = pd.to_datetime(df_d['YEAR'] + '-' + df_d['MONTH'] + '-01')

# Perform the join
joined_df = pd.merge(general_cpi_df, df_d, left_on='Date (#)', right_on='DATE', how='inner')

# Display the result
joined_df

,EXTRACTIONDATE,REG,Descripción series,Date (varchar),Date (#),IPC,YEAR,MONTH,LIKELIHOODOFDEFAULT,DATE
0,2024-03-21 21:55:32.380,1.0,IPC general (empalme BCCh),Jan.2009,2009-01-01,59.05,2009,01,0.052036,2009-01-01
1,2024-03-21 21:55:32.380,1.0,IPC general (empalme BCCh),Feb.2009,2009-02-01,58.28,2009,02,0.050888,2009-02-01
2,2024-03-21 21:55:32.380,1.0,IPC general (empalme BCCh),Mar.2009,2009-03-01,58.30,2009,03,0.050589,2009-03-01
3,2024-03-21 21:55:32.380,1.0,IPC general (empalme BCCh),Apr.2009,2009-04-01,58.21,2009,04,0.058679,2009-04-01
4,2024-03-21 21:55:32.380,1.0,IPC general (empalme BCCh),May.2009,2009-05-01,58.09,2009,05,0.065349,2009-05-01
...,...,...,...,...,...,...,...,...,...,...
176,2024-03-21 21:55:32.380,1.0,IPC general (empalme BCCh),Sep.2023,2023-09-01,100.53,2023,09,0.293259,2023-09-01
177,2024-03-21 21:55:32.380,1.0,IPC general (empalme BCCh),Oct.2023,2023-10-01,100.82,2023,10,0.267782,2023-10-01
178,2024-03-21 21:55:32.380,1.0,IPC general (empalme BCCh),Nov.2023,2023-11-01,101.58,2023,11,0.360104,2023-11-01
179,2024-03-21 21:55:32.380,1.0,IPC general (empalme BCCh),Dec.2023,2023-12-01,101.04,2023,12,0.439466,2023-12-01


In [ ]:
joined_df.to_csv('general_cpi_monthly_agg.csv', index=False)
files.download('general_cpi_monthly_agg.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Merging Tables: Santiago GDP, yearly

In [ ]:
df_gdp_s = session.sql('select * from GDP_S_COPY')
df_gdp_s = df_gdp_s.select([F.col(col_name).alias(col_name.replace('"', '')) for col_name in df_gdp_s.columns])
df_gdp_s = df_gdp_s[df_gdp_s['"Descripción series"'] == 'Producto Interno Bruto']

df_d=session.sql('select * from YEARLY_DEFAULT ')
df_d = df_d.select([F.col(col_name).alias(col_name.replace('"', '')) for col_name in df_d.columns])

In [ ]:
df_gdp_s=df_gdp_s.to_pandas()
df_d=df_d.to_pandas()

In [ ]:
df_gdp_s

,EXTRACTIONDATE,REG,Descripción series,YEAR,DATE,GDP(million_pesos)
0,2024-03-23,14.0,Producto Interno Bruto,2013.0,2013-12-31,57950.692393
1,2024-03-23,14.0,Producto Interno Bruto,2014.0,2014-12-31,61747.513018
2,2024-03-23,14.0,Producto Interno Bruto,2015.0,2015-12-31,67395.594568
3,2024-03-23,14.0,Producto Interno Bruto,2016.0,2016-12-31,72096.318083
4,2024-03-23,14.0,Producto Interno Bruto,2017.0,2017-12-31,75041.892228
5,2024-03-23,14.0,Producto Interno Bruto,2018.0,2018-12-31,79326.313684
6,2024-03-23,14.0,Producto Interno Bruto,2019.0,2019-12-31,82149.481470
7,2024-03-23,14.0,Producto Interno Bruto,2020.0,2020-12-31,79602.544458
8,2024-03-23,14.0,Producto Interno Bruto,2021.0,2021-12-31,91828.491152
9,2024-03-23,14.0,Producto Interno Bruto,2022.0,2022-12-31,99978.597924


In [ ]:
df_d

,YEAR,LIKELIHOODOFDEFAULT
0,2014,0.074191
1,2006,0.032722
2,2007,0.048319
3,2008,0.057514
4,2023,0.265869
5,2024,0.754162
6,2021,0.153499
7,2022,0.181699
8,2009,0.065448
9,2015,0.081544


In [ ]:
# Convert YEAR column to integer in both DataFrames if necessary
df_gdp_s['YEAR'] = df_gdp_s['YEAR'].astype(int)
df_d['YEAR'] = df_d['YEAR'].astype(int)

# Perform the merge
merged_df = pd.merge(df_gdp_s, df_d, on='YEAR', how='inner')

# Display the result
merged_df


,EXTRACTIONDATE,REG,Descripción series,YEAR,DATE,GDP(million_pesos),LIKELIHOODOFDEFAULT
0,2024-03-23,14.0,Producto Interno Bruto,2013,2013-12-31,57950.692393,0.068015
1,2024-03-23,14.0,Producto Interno Bruto,2014,2014-12-31,61747.513018,0.074191
2,2024-03-23,14.0,Producto Interno Bruto,2015,2015-12-31,67395.594568,0.081544
3,2024-03-23,14.0,Producto Interno Bruto,2016,2016-12-31,72096.318083,0.099478
4,2024-03-23,14.0,Producto Interno Bruto,2017,2017-12-31,75041.892228,0.099008
5,2024-03-23,14.0,Producto Interno Bruto,2018,2018-12-31,79326.313684,0.096684
6,2024-03-23,14.0,Producto Interno Bruto,2019,2019-12-31,82149.481470,0.108227
7,2024-03-23,14.0,Producto Interno Bruto,2020,2020-12-31,79602.544458,0.145762
8,2024-03-23,14.0,Producto Interno Bruto,2021,2021-12-31,91828.491152,0.153499
9,2024-03-23,14.0,Producto Interno Bruto,2022,2022-12-31,99978.597924,0.181699


In [ ]:
merged_df.to_csv('Santiago_gdp_yearly_agg.csv', index=False)
files.download('Santiago_gdp_yearly_agg.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Aggregated Household debt, yearly

In [ ]:
df_debt = session.sql('select * from HOUSEHOLD_DEBT_COPY')
df_debt = df_debt.select([F.col(col_name).alias(col_name.replace('"', '')) for col_name in df_debt.columns])

df_d=session.sql('select * from YEARLY_DEFAULT ')
df_d = df_d.select([F.col(col_name).alias(col_name.replace('"', '')) for col_name in df_d.columns])

In [ ]:
df_debt=df_debt.to_pandas()
df_d=df_d.to_pandas()

In [ ]:
df_debt

,EXTRACTIONDATE,COUNTRY,YEAR,DATE,Debt(%GDP)
0,2024-03-23,Chile,2002.0,2002-12-31,22.630873
1,2024-03-23,Chile,2003.0,2003-12-31,23.503435
2,2024-03-23,Chile,2004.0,2004-12-31,25.165786
3,2024-03-23,Chile,2005.0,2005-12-31,26.329949
4,2024-03-23,Chile,2006.0,2006-12-31,26.392439
5,2024-03-23,Chile,2007.0,2007-12-31,29.027133
6,2024-03-23,Chile,2008.0,2008-12-31,32.631886
7,2024-03-23,Chile,2009.0,2009-12-31,33.905979
8,2024-03-23,Chile,2010.0,2010-12-31,32.583076
9,2024-03-23,Chile,2011.0,2011-12-31,33.805600


In [ ]:
# Convert YEAR column to integer in both DataFrames
df_debt['YEAR'] = df_debt['YEAR'].astype(int)
df_d['YEAR'] = df_d['YEAR'].astype(int)

# Perform the merge
merged_df = pd.merge(df_debt, df_d, on='YEAR', how='inner')


In [ ]:
merged_df.to_csv('Household_debt_yearly_agg.csv', index=False)
files.download('Household_debt_yearly_agg.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Transportation Cost Index, aggregated monthly

In [ ]:
df_tci = session.sql('select * from TCI_COPY')
df_tci = df_tci.select([F.col(col_name).alias(col_name.replace('"', '')) for col_name in df_tci.columns])

df_d=session.sql('select * from MONTHLY_DEFAULT ')
df_d = df_d.select([F.col(col_name).alias(col_name.replace('"', '')) for col_name in df_d.columns])

In [ ]:
df_tci=df_tci.to_pandas()
df_d=df_d.to_pandas()

In [ ]:
df_tci

,EXTRACTIONDATE,DATE,YEAR,MONTH,GLOSA,INDEX,Monthly_variation(%)
0,2024-03-23,2019-01-01,2019.0,1.0,ICT Sin Diésel,103.41,1.9
1,2024-03-23,2019-02-01,2019.0,2.0,ICT Sin Diésel,101.26,-2.1
2,2024-03-23,2019-03-01,2019.0,3.0,ICT Sin Diésel,99.60,-1.6
3,2024-03-23,2019-04-01,2019.0,4.0,ICT Sin Diésel,99.95,0.4
4,2024-03-23,2019-05-01,2019.0,5.0,ICT Sin Diésel,102.00,2.0
5,2024-03-23,2019-06-01,2019.0,6.0,ICT Sin Diésel,101.84,-0.2
6,2024-03-23,2019-07-01,2019.0,7.0,ICT Sin Diésel,101.15,-0.7
7,2024-03-23,2019-08-01,2019.0,8.0,ICT Sin Diésel,101.90,0.7
8,2024-03-23,2019-09-01,2019.0,9.0,ICT Sin Diésel,101.98,0.1
9,2024-03-23,2019-10-01,2019.0,10.0,ICT Sin Diésel,103.67,1.7


In [ ]:
df_d

,YEAR,MONTH,LIKELIHOODOFDEFAULT
0,2010,7,0.096109
1,2010,8,0.096293
2,2010,9,0.098058
3,2010,10,0.098337
4,2010,11,0.092078
...,...,...,...
212,2010,2,0.091179
213,2010,3,0.092471
214,2010,4,0.101637
215,2010,5,0.103590


In [ ]:
# Convert YEAR and MONTH to integer in both DataFrames if necessary
df_tci['YEAR'] = df_tci['YEAR'].astype(int)
df_tci['MONTH'] = df_tci['MONTH'].astype(int)
df_d['YEAR'] = df_d['YEAR'].astype(int)
df_d['MONTH'] = df_d['MONTH'].astype(int)

# Perform the merge
merged_df = pd.merge(df_tci, df_d, on=['YEAR', 'MONTH'], how='inner')

# Display the result
merged_df


,EXTRACTIONDATE,DATE,YEAR,MONTH,GLOSA,INDEX,Monthly_variation(%),LIKELIHOODOFDEFAULT
0,2024-03-23,2019-01-01,2019,1,ICT Sin Diésel,103.41,1.9,0.091579
1,2024-03-23,2019-02-01,2019,2,ICT Sin Diésel,101.26,-2.1,0.098561
2,2024-03-23,2019-03-01,2019,3,ICT Sin Diésel,99.60,-1.6,0.098208
3,2024-03-23,2019-04-01,2019,4,ICT Sin Diésel,99.95,0.4,0.096204
4,2024-03-23,2019-05-01,2019,5,ICT Sin Diésel,102.00,2.0,0.103981
5,2024-03-23,2019-06-01,2019,6,ICT Sin Diésel,101.84,-0.2,0.106122
6,2024-03-23,2019-07-01,2019,7,ICT Sin Diésel,101.15,-0.7,0.105246
7,2024-03-23,2019-08-01,2019,8,ICT Sin Diésel,101.90,0.7,0.117354
8,2024-03-23,2019-09-01,2019,9,ICT Sin Diésel,101.98,0.1,0.116293
9,2024-03-23,2019-10-01,2019,10,ICT Sin Diésel,103.67,1.7,0.106816


In [ ]:
merged_df.to_csv('TCI_monthly_agg.csv', index=False)
files.download('TCI_monthly_agg.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Highway Santiego, aggregated yearly

In [ ]:
df_highway = session.sql('select * from TRAFFIC_SANTIAGO_COPY')
df_highway = df_highway.select([F.col(col_name).alias(col_name.replace('"', '')) for col_name in df_highway.columns])
df_highway = df_highway.filter(df_highway['Descripción series'] == "Pasada de vehículos por pórticos de autopistas urbanas, unidades")

df_d=session.sql('select * from YEARLY_DEFAULT ')
df_d = df_d.select([F.col(col_name).alias(col_name.replace('"', '')) for col_name in df_d.columns])

In [ ]:
df_highway=df_highway.to_pandas()
df_d=df_d.to_pandas()

In [ ]:
df_highway

,EXTRACTIONDATE,REG,Descripción series,YEAR,DATE,INDEX
0,2024-03-23,16.0,Pasada de vehículos por pórticos de autopistas...,2014.0,2014-12-31,9.905555e+07
1,2024-03-23,16.0,Pasada de vehículos por pórticos de autopistas...,2015.0,2015-12-31,1.073556e+08
2,2024-03-23,16.0,Pasada de vehículos por pórticos de autopistas...,2016.0,2016-12-31,1.147990e+08
3,2024-03-23,16.0,Pasada de vehículos por pórticos de autopistas...,2017.0,2017-12-31,1.208203e+08
4,2024-03-23,16.0,Pasada de vehículos por pórticos de autopistas...,2018.0,2018-12-31,1.234694e+08
5,2024-03-23,16.0,Pasada de vehículos por pórticos de autopistas...,2019.0,2019-12-31,1.260993e+08
6,2024-03-23,16.0,Pasada de vehículos por pórticos de autopistas...,2020.0,2020-12-31,9.553361e+07
7,2024-03-23,16.0,Pasada de vehículos por pórticos de autopistas...,2021.0,2021-12-31,1.229161e+08
8,2024-03-23,16.0,Pasada de vehículos por pórticos de autopistas...,2022.0,2022-12-31,1.327345e+08
9,2024-03-23,16.0,Pasada de vehículos por pórticos de autopistas...,2023.0,2023-12-31,1.311635e+08


In [ ]:
df_d

,YEAR,LIKELIHOODOFDEFAULT
0,2014,0.074191
1,2006,0.032722
2,2007,0.048319
3,2008,0.057514
4,2023,0.265869
5,2024,0.754162
6,2021,0.153499
7,2022,0.181699
8,2009,0.065448
9,2015,0.081544


In [ ]:
# Convert the YEAR column to integer in both DataFrames
df_highway['YEAR'] = df_highway['YEAR'].astype(int)
df_d['YEAR'] = df_d['YEAR'].astype(int)

# Perform the merge
merged_df = pd.merge(df_highway, df_d, on='YEAR', how='inner')

# Display the result
merged_df

,EXTRACTIONDATE,REG,Descripción series,YEAR,DATE,INDEX,LIKELIHOODOFDEFAULT
0,2024-03-23,16.0,Pasada de vehículos por pórticos de autopistas...,2014,2014-12-31,9.905555e+07,0.074191
1,2024-03-23,16.0,Pasada de vehículos por pórticos de autopistas...,2015,2015-12-31,1.073556e+08,0.081544
2,2024-03-23,16.0,Pasada de vehículos por pórticos de autopistas...,2016,2016-12-31,1.147990e+08,0.099478
3,2024-03-23,16.0,Pasada de vehículos por pórticos de autopistas...,2017,2017-12-31,1.208203e+08,0.099008
4,2024-03-23,16.0,Pasada de vehículos por pórticos de autopistas...,2018,2018-12-31,1.234694e+08,0.096684
5,2024-03-23,16.0,Pasada de vehículos por pórticos de autopistas...,2019,2019-12-31,1.260993e+08,0.108227
6,2024-03-23,16.0,Pasada de vehículos por pórticos de autopistas...,2020,2020-12-31,9.553361e+07,0.145762
7,2024-03-23,16.0,Pasada de vehículos por pórticos de autopistas...,2021,2021-12-31,1.229161e+08,0.153499
8,2024-03-23,16.0,Pasada de vehículos por pórticos de autopistas...,2022,2022-12-31,1.327345e+08,0.181699
9,2024-03-23,16.0,Pasada de vehículos por pórticos de autopistas...,2023,2023-12-31,1.311635e+08,0.265869


In [ ]:
merged_df.to_csv('highway_yearly_agg.csv', index=False)
files.download('highway_yearly_agg.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Santiego Highway interurban yearly


In [ ]:
df_highway = session.sql('select * from TRAFFIC_SANTIAGO_COPY')
df_highway = df_highway.select([F.col(col_name).alias(col_name.replace('"', '')) for col_name in df_highway.columns])
df_highway = df_highway.filter(df_highway['Descripción series'] == "Pasada de vehículos por plazas de peajes y pórticos de autopistas interurbanas, unidades")

df_d=session.sql('select * from YEARLY_DEFAULT ')
df_d = df_d.select([F.col(col_name).alias(col_name.replace('"', '')) for col_name in df_d.columns])

In [ ]:
df_highway=df_highway.to_pandas()
df_d=df_d.to_pandas()

In [ ]:
# Convert the YEAR column to integer in both DataFrames
df_highway['YEAR'] = df_highway['YEAR'].astype(int)
df_d['YEAR'] = df_d['YEAR'].astype(int)

# Perform the merge
merged_df = pd.merge(df_highway, df_d, on='YEAR', how='inner')

# Display the result
merged_df

,EXTRACTIONDATE,REG,Descripción series,YEAR,DATE,INDEX,LIKELIHOODOFDEFAULT
0,2024-03-23,15.0,Pasada de vehículos por plazas de peajes y pór...,2014,2014-12-31,9.557303e+06,0.074191
1,2024-03-23,15.0,Pasada de vehículos por plazas de peajes y pór...,2015,2015-12-31,1.048325e+07,0.081544
2,2024-03-23,15.0,Pasada de vehículos por plazas de peajes y pór...,2016,2016-12-31,1.182673e+07,0.099478
3,2024-03-23,15.0,Pasada de vehículos por plazas de peajes y pór...,2017,2017-12-31,1.352149e+07,0.099008
4,2024-03-23,15.0,Pasada de vehículos por plazas de peajes y pór...,2018,2018-12-31,1.735575e+07,0.096684
5,2024-03-23,15.0,Pasada de vehículos por plazas de peajes y pór...,2019,2019-12-31,2.167174e+07,0.108227
6,2024-03-23,15.0,Pasada de vehículos por plazas de peajes y pór...,2020,2020-12-31,1.624858e+07,0.145762
7,2024-03-23,15.0,Pasada de vehículos por plazas de peajes y pór...,2021,2021-12-31,2.206913e+07,0.153499
8,2024-03-23,15.0,Pasada de vehículos por plazas de peajes y pór...,2022,2022-12-31,2.501330e+07,0.181699
9,2024-03-23,15.0,Pasada de vehículos por plazas de peajes y pór...,2023,2023-12-31,2.450204e+07,0.265869


In [ ]:
merged_df.to_csv('highway_interurban_yearly_agg.csv', index=False)
files.download('highway_interurban_yearly_agg.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

GDP Deflator Yearly Aggregated

In [ ]:
df_deflator = session.sql('select * from GDP_DEFLATOR_COPY')
df_deflator = df_deflator.select([F.col(col_name).alias(col_name.replace('"', '')) for col_name in df_deflator.columns])

df_d=session.sql('select * from YEARLY_DEFAULT ')
df_d = df_d.select([F.col(col_name).alias(col_name.replace('"', '')) for col_name in df_d.columns])

In [ ]:
df_deflator=df_deflator.to_pandas()
df_d=df_d.to_pandas()

In [ ]:
df_deflator

,EXTRACTIONDATE,Country Name,YEAR,DATE,deflator(annual%)
0,2024-03-23,Chile,1960.0,1960-12-31,NaN
1,2024-03-23,Chile,1961.0,1961-12-31,6.568982
2,2024-03-23,Chile,1962.0,1962-12-31,12.948516
3,2024-03-23,Chile,1963.0,1963-12-31,43.820427
4,2024-03-23,Chile,1964.0,1964-12-31,47.934362
...,...,...,...,...,...
58,2024-03-23,Chile,2018.0,2018-12-31,1.590200
59,2024-03-23,Chile,2019.0,2019-12-31,2.572598
60,2024-03-23,Chile,2020.0,2020-12-31,9.637414
61,2024-03-23,Chile,2021.0,2021-12-31,6.797841


In [ ]:
# Convert the YEAR column to integer in both DataFrames
df_deflator['YEAR'] = df_deflator['YEAR'].astype(int)
df_d['YEAR'] = df_d['YEAR'].astype(int)

# Perform the inner join merge
merged_df = pd.merge(df_deflator, df_d, on='YEAR', how='inner')

# Display the result
merged_df


,EXTRACTIONDATE,Country Name,YEAR,DATE,deflator(annual%),LIKELIHOODOFDEFAULT
0,2024-03-23,Chile,2006,2006-12-31,12.349896,0.032722
1,2024-03-23,Chile,2007,2007-12-31,5.088757,0.048319
2,2024-03-23,Chile,2008,2008-12-31,0.311133,0.057514
3,2024-03-23,Chile,2009,2009-12-31,3.577795,0.065448
4,2024-03-23,Chile,2010,2010-12-31,8.857441,0.094427
5,2024-03-23,Chile,2011,2011-12-31,3.260515,0.101237
6,2024-03-23,Chile,2012,2012-12-31,0.763467,0.107576
7,2024-03-23,Chile,2013,2013-12-31,2.260771,0.068015
8,2024-03-23,Chile,2014,2014-12-31,5.852892,0.074191
9,2024-03-23,Chile,2015,2015-12-31,4.954366,0.081544


In [ ]:
merged_df.to_csv('deflator_yearly_agg.csv', index=False)
files.download('deflator_yearly_agg.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

GDP Chile normalized, Aggregated Monthly

In [ ]:
df_gdp = session.sql('select * from GDP_NORM_COPY')
df_gdp = df_gdp.select([F.col(col_name).alias(col_name.replace('"', '')) for col_name in df_gdp.columns])

df_d=session.sql('select * from MONTHLY_DEFAULT ')
df_d = df_d.select([F.col(col_name).alias(col_name.replace('"', '')) for col_name in df_d.columns])

In [ ]:
df_gdp=df_gdp.to_pandas()
df_d=df_d.to_pandas()

In [ ]:
df_gdp

,EXTRACTIONDATE,OBSERVATION_DATE,GDP_NORMALIZED
0,2024-03-31,1995-02-01,98.273906
1,2024-03-31,1995-03-01,98.426386
2,2024-03-31,1995-04-01,98.565048
3,2024-03-31,1995-05-01,98.673092
4,2024-03-31,1995-06-01,98.743644
...,...,...,...
317,2024-03-31,2021-07-01,100.708930
318,2024-03-31,2021-08-01,101.208495
319,2024-03-31,2021-09-01,101.663892
320,2024-03-31,2021-10-01,102.068399


In [ ]:
# Convert OBSERVATION_DATE to datetime if it's not already
df_gdp['OBSERVATION_DATE'] = pd.to_datetime(df_gdp['OBSERVATION_DATE'])

# Extract year and month from OBSERVATION_DATE in df_gdp
df_gdp['YEAR'] = df_gdp['OBSERVATION_DATE'].dt.year
df_gdp['MONTH'] = df_gdp['OBSERVATION_DATE'].dt.month

# Convert YEAR and MONTH in df_d to integer if they are not already
df_d['YEAR'] = df_d['YEAR'].astype(int)
df_d['MONTH'] = df_d['MONTH'].astype(int)

# Perform the inner join merge
merged_df = pd.merge(df_gdp, df_d, on=['YEAR', 'MONTH'], how='inner')

# Display the result
merged_df

,EXTRACTIONDATE,OBSERVATION_DATE,GDP_NORMALIZED,YEAR,MONTH,LIKELIHOODOFDEFAULT
0,2024-03-31,2006-01-01,100.776983,2006,1,0.019363
1,2024-03-31,2006-02-01,100.804387,2006,2,0.017998
2,2024-03-31,2006-03-01,100.858167,2006,3,0.036930
3,2024-03-31,2006-04-01,100.934271,2006,4,0.036691
4,2024-03-31,2006-05-01,101.023159,2006,5,0.026561
...,...,...,...,...,...,...
186,2024-03-31,2021-07-01,100.708930,2021,7,0.144391
187,2024-03-31,2021-08-01,101.208495,2021,8,0.166013
188,2024-03-31,2021-09-01,101.663892,2021,9,0.157074
189,2024-03-31,2021-10-01,102.068399,2021,10,0.139014


In [ ]:
merged_df.to_csv('gdp_norm_monthly_agg.csv', index=False)
files.download('gdp_norm_monthly_agg.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Unemployment aggregated yearly

In [ ]:
df_unemployment = session.sql('select * from UNEMPLOYMENT_COPY')
df_unemployment = df_unemployment.select([F.col(col_name).alias(col_name.replace('"', '')) for col_name in df_unemployment.columns])

df_d=session.sql('select * from YEARLY_DEFAULT ')
df_d = df_d.select([F.col(col_name).alias(col_name.replace('"', '')) for col_name in df_d.columns])

In [ ]:
df_unemployment=df_unemployment.to_pandas()
df_d=df_d.to_pandas()

In [ ]:
df_unemployment

,EXTRACTIONDATE,COUNTRY,DATE,YEAR,Rate(percent)
0,2024-03-22 12:51:02.513,Chile,1980-12-31,1980.0,11.5
1,2024-03-22 12:51:02.513,Chile,1981-12-31,1981.0,10.3
2,2024-03-22 12:51:02.513,Chile,1982-12-31,1982.0,19.8
3,2024-03-22 12:51:02.513,Chile,1983-12-31,1983.0,21.0
4,2024-03-22 12:51:02.513,Chile,1984-12-31,1984.0,17.5
5,2024-03-22 12:51:02.513,Chile,1985-12-31,1985.0,15.0
6,2024-03-22 12:51:02.513,Chile,1986-12-31,1986.0,12.3
7,2024-03-22 12:51:02.513,Chile,1987-12-31,1987.0,11.0
8,2024-03-22 12:51:02.513,Chile,1988-12-31,1988.0,9.9
9,2024-03-22 12:51:02.513,Chile,1989-12-31,1989.0,8.0


In [ ]:
# Convert the YEAR column to integer in both DataFrames
df_unemployment['YEAR'] = df_unemployment['YEAR'].astype(int)
df_d['YEAR'] = df_d['YEAR'].astype(int)

# Perform the inner join merge
merged_df = pd.merge(df_unemployment, df_d, on='YEAR', how='inner')

merged_df


,EXTRACTIONDATE,COUNTRY,DATE,YEAR,Rate(percent),LIKELIHOODOFDEFAULT
0,2024-03-22 12:51:02.513,Chile,2006-12-31,2006,8.0,0.032722
1,2024-03-22 12:51:02.513,Chile,2007-12-31,2007,7.0,0.048319
2,2024-03-22 12:51:02.513,Chile,2008-12-31,2008,7.8,0.057514
3,2024-03-22 12:51:02.513,Chile,2009-12-31,2009,11.1,0.065448
4,2024-03-22 12:51:02.513,Chile,2010-12-31,2010,8.3,0.094427
5,2024-03-22 12:51:02.513,Chile,2011-12-31,2011,7.3,0.101237
6,2024-03-22 12:51:02.513,Chile,2012-12-31,2012,6.6,0.107576
7,2024-03-22 12:51:02.513,Chile,2013-12-31,2013,6.1,0.068015
8,2024-03-22 12:51:02.513,Chile,2014-12-31,2014,6.5,0.074191
9,2024-03-22 12:51:02.513,Chile,2015-12-31,2015,6.3,0.081544


In [ ]:
merged_df.to_csv('unemployment_yearly_agg.csv', index=False)
files.download('unemployment_yearly_agg.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
session.close

<bound method Session.close of <snowflake.snowpark.session.Session object at 0x7fa5b7534790>>

In [ ]:
s